# 第3部 第8章 ポアソン回帰モデル

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
cmap = plt.get_cmap("tab10")
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 8.1 本章の目的と概要
参考書籍から引用します。\
RとStanやbrmsの部分をPythonとNumPyroに読み替えてください。

> **テーマ**\
> 本章ではポアソン回帰モデルの推定を行います.

> **目的**\
> 一般化線形モデルにおいて,線形予測子をいろいろと変えていく手順は,第2章の単回帰モデルから第6章の分散分析モデル(ダミー変数),そして第7章の正規線形モデルで説明しました.\
> 続いて本章では,確率分布とリンク関数を変更する手続きを体験して頂きます.

> **概要**\
> モデルの構造 → 分析の準備 → データの読み込みと可視化\
> → brms によるポアソン回帰モデルの推定 → 推定されたモデルの解釈 → 回帰曲線の図示\
> → (補足)Stanファイルの実装 → (補足)デザイン行列を使ったStanファイルの実装

## 8.2 モデルの実装
湖で1時間釣りをしたときの釣獲尾数 $y_{i}$ と天気と気温の組 $( x_{1}, x_{2} )$ との関係をモデル化します。\
書籍のモデル構造式を掲載します。\
ただしモデルの考えやすさの都合上ギリシャ文字を一部変更しました。\
ここで天気 $x_{1}$ は晴れの日に $1$ ,そうでない日に $0$ を取る二値変数です。\
気温 $x_{2}$ は連続値変数です。

$$
\begin{aligned}
\mu_{i} =& \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2}\\
y_{i} \sim& Poiss \left ( \exp (\mu_{i})  \right )
\end{aligned}
$$

この構造式に至るための仮定を補足します。\
釣獲尾数 $y_{i}$ は $0$ 以上の整数なので $y_{i}$ は離散的な確率分布に従います。\
さらに1時間という短い時間なので $y_{i}$ は $0$ に近い整数値を取るのが自然でしょう。\
これらの条件を満たす確率分布としてポアソン分布を選びます。\
ポアソン分布は正の実数 $\lambda_{i}$ をパラメータに持つ離散的な確率分布です。

$$
\begin{aligned}
y_{i} \sim& Poiss \left ( \lambda_{i}  \right )\\
\lambda_{i} >& 0
\end{aligned}
$$

天気と気温の組の影響をモデル化したいので1次関数 $\mu_{i} = \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2}$ を確率分布に組込みたいです。\
しかし1次関数近似は負の数を返す可能性があり確率分布が成立しない危険性があります。\
指数関数に1次関数を渡した値をポアソン分布のパラメータにすることで、この悩みを解決します。

$$
\begin{aligned}
y_{i} \sim& Poiss \left ( \lambda_{i}  \right )\\
\lambda_{i} \equiv& \exp \left( \mu_{i} \right)\\
=& \exp \left( \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2} \right)
\end{aligned}
$$

湖で1時間釣りをしたときの釣獲尾数 $y_{i}$ と天気と気温の組 $( x_{1}, x_{2} )$ との関係をモデル化できました。

## 8.3 分析の準備
ライブラリの準備をしている節です。\
省略します。

## 8.4 データの読み込みと可視化
データを読み込んで概要を把握します。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-8-1-fish-num-1.csv")
display(df.head())

# サンプルサイズを表示する
print( len(df) )

釣獲尾数と気温の関係の散布図を天気で層別してプロットします。\
気温が高いほど釣獲尾数の平均値と分散が大きくなっています。\
ポアソン分布で近似してもよさそうです。

In [ ]:
ax = sns.scatterplot(data = df, x = "temperature", y = "fish_num", hue = "weather")
ax.set_title("図3.8.1 釣獲尾数と天気・気温の散布図")

モデルの構造の通りのダミー変数を作成します.

In [ ]:
df = (
    df
    .with_columns([
        pl.when( pl.col("weather") == "sunny" ).then(1)
        .otherwise(0)
        .alias("x1"),
    ])
)
display(df.head())

numpyroに渡すためにjax.numpy配列に変換します。

In [ ]:
X1 = jnp.array(df["x1"], dtype = int)
X2 = jnp.array(df["temperature"], dtype = float)
Y = jnp.array(df["fish_num"], dtype = int)

## 8.5 brmsによるポアソン回帰モデルの推定
NumPyroでモデルを作成します。

In [ ]:
def model_poisson(X1, X2, Y = None):
    '''
        第3部 8章 の釣獲尾数
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X1)
    with numpyro.plate("N", N):
        X1 = numpyro.deterministic("X1", X1)
        X2 = numpyro.deterministic("X2", X2)
    # 確率変数のパラメータの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 100))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 10))
    β2 = numpyro.sample("β2", dist.Normal(loc = 0, scale = 10))
    # リンク関数を定義する
    with numpyro.plate("N", N):
        μ = numpyro.deterministic("μ", β0 + β1 * X1 + β2 * X2)
        λ = numpyro.deterministic("λ", jnp.exp(μ))
    # 目的変数 y は説明変数 x の値に応じた値 λ をパラメータとするポアソン分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Poisson(rate = λ), obs = Y)

モデルをプロットします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y,
}
try_render_model(model_poisson, render_name = "釣獲尾数と天気・気温の関係", **model_args)

ユーティリティ関数にモデルを渡してサンプリングします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "Y": Y
}
mcmc = run_mcmc(
    model_poisson,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    **model_args
)
idata = az.from_numpyro(mcmc, log_likelihood = False)

サンプリングが上手くいったか確認します。

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["β0", "β1", "β2"])
plt.tight_layout()

MCMCサンプルの概要を確認します。\
HDIを見ると $\beta 1$ が負の値なので晴れの日は釣獲尾数が減り、 $\beta 2$ が正の数なので気温が高いほどよく釣れるようです。

In [ ]:
az.summary(idata, var_names = ["β0", "β1", "β2"])

## 8.6 推定されたモデルの解釈
$\lambda = \exp \left( \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2} \right)$ であることに注意します。\
晴れの日は平均が $\exp \left( \beta_{1} \right) = \exp (-0.598) \approx 0.55$ 倍されます。

## 8.7 回帰曲線の図示
天気別の事後予測分布を計算します。\
NumPyroは少し準備が必要です。

In [ ]:
# 予測用のデータ
list_temperature = list( range( int(df["temperature"].floor().min()), int(df["temperature"].ceil().max()) ) )
X2 = jnp.array(list_temperature * 2, dtype = float)
N_X = len(list_temperature)
model_args = {
    "X1": jnp.array([0] * N_X + [1] * N_X, dtype = int),
    "X2": X2,
    "Y": None,
}
# 予測データの目的変数名
var_name = "Y"

# 事後予測分布を計算する
idata = compute_posterior_predictive_distribution(
    model_poisson,
    mcmc,
    var_name = "Y",
    seed = 42,
    hdi_prob = 0.95,
    log_likelihood = False,
    **model_args,
)

# 事後予測平均
y_ppc_mean = np.array( idata["posterior_predictive"][var_name].mean(dim = ("chain", "draw")) )
# 事後予測のHDI
hdi_ppc = az.hdi(
    idata,                          # InferenceData 全体を渡す
    group = "posterior_predictive", # 計算対象グループを指定
    var_names = [var_name],         # 対象変数
    hdi_prob = 0.95,            # 区間幅
)
y_ppc_low = np.array( hdi_ppc[var_name][:, 0] )
y_ppc_high = np.array( hdi_ppc[var_name][:, 1] )

# データを保存する
df_pred = pl.DataFrame({
    "weather": ["cloudy"] * N_X + ["sunny"] * N_X,
    "temperature": np.array(X2),
    "y_pred": y_ppc_mean,
    "y_ppc_low": y_ppc_low,
    "y_ppc_high": y_ppc_high,
})

散布図と事後予測分布をプロットします。\
晴れの日の売り上げが好調であることがわかります。

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1)
for idx, weather in enumerate(df["weather"].unique().sort()):
    expr = pl.col("weather") == weather
    sns.scatterplot(data = df.filter(expr), x = "temperature", y = "fish_num", ax = ax, color = cmap(idx), label = weather)
    sns.lineplot(data = df_pred.filter(expr), x = "temperature", y = "y_pred", ax = ax, color = cmap(idx))
    ax.fill_between(
        x = df_pred.filter(expr)["temperature"], y1 = df_pred.filter(expr)["y_ppc_high"], y2 = df_pred.filter(expr)["y_ppc_low"],
        color = cmap(idx), alpha=0.2
    )
plt.suptitle("図3.8.2 ポアソン回帰曲線: 信用区間")

## 8.8 補足: ポアソン回帰モデルのためのStanファイルの実装
省略します。

## 8.9 補足: ポアソン回帰モデルのためのStanファイルの実装(デザイン行列使用)
省略します。

## 終わりに